<font color='purple' size=4><b>Importing Libraries</b></font>

In [1]:
import cv2
import numpy as np
import mediapipe as mp
import pandas as pd
import os
import warnings

<font color='purple' size=4><b>Initialising Pose Detection Model</b></font>

In [2]:
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=True, min_detection_confidence=0.3, model_complexity=2)

<font color='purple' size=4><b>Creating List With Names of All Poses</b></font>

In [3]:
pose_list = ['Downward Dog Pose', 'Goddess Pose', 'Plank Pose', 'Tree Pose', 'Warrior-II Pose']

<font color='purple' size=4><b>Creating Pose Detection Function</b></font>

In [4]:
def get_landmarks(image_path, pose):
    image = cv2.imread(image_path)
    landmarks_list = []
    if image.ndim == 3:
        if image.shape[-1] == 3:            
            imageRGB = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            results = pose.process(imageRGB)
            height, width, _ = image.shape

            if results.pose_landmarks:        
                for landmark in results.pose_landmarks.landmark:
                    landmarks_list.append((int(landmark.x*width), int(landmark.y*height), (landmark.z*width)))

                return landmarks_list

            else:
                return
    return

<font color='purple' size=4><b>Creating Angle Calculation Function</b></font>

In [5]:
def get_angle(landmark1, landmark2, landmark3):
    warnings.filterwarnings('error')
    
    x1, y1, _ = landmark1
    x2, y2, _ = landmark2
    x3, y3, _ = landmark3

    pt1 = np.array([x1, y1])
    pt2 = np.array([x2, y2])
    pt3 = np.array([x3, y3])

    line21 = pt1 - pt2
    line23 = pt3 - pt2

    cosine_angle = np.dot(line21, line23) / (np.linalg.norm(line21) * np.linalg.norm(line23))
    try:
        angle_rad = np.arccos(cosine_angle)
        angle_deg = np.degrees(angle_rad)
        
    except:
        return
    
    if angle_deg < 0:
        return (360 + angle_deg)
    
    else: 
        return angle_deg

<font color='purple' size=4><b>Creating Function to Obtain Features From Landmarks</b></font>

In [6]:
def get_features(landmarks_list):
    shoulder_ang_L = get_angle(landmarks_list[13], landmarks_list[11], landmarks_list[23])
    shoulder_ang_R = get_angle(landmarks_list[14], landmarks_list[12], landmarks_list[24])
    elbow_ang_L = get_angle(landmarks_list[11], landmarks_list[13], landmarks_list[15])
    elbow_ang_R = get_angle(landmarks_list[12], landmarks_list[14], landmarks_list[16])
    waist_ang_L = get_angle(landmarks_list[11], landmarks_list[23], landmarks_list[25])
    waist_ang_R = get_angle(landmarks_list[12], landmarks_list[24], landmarks_list[26])
    knee_ang_L = get_angle(landmarks_list[23], landmarks_list[25], landmarks_list[27])
    knee_ang_R = get_angle(landmarks_list[24], landmarks_list[26], landmarks_list[28])
    
    features = [shoulder_ang_L, shoulder_ang_R, elbow_ang_L, elbow_ang_R,
                waist_ang_L, waist_ang_R, knee_ang_L, knee_ang_R]
    
    features = np.array(features)
    return features

<font color='purple' size=4><b>Creating Preprocess Function</b></font>

In [7]:
def preprocess(path):
    main_folder = os.path.dirname(os.path.dirname(path))
    csv_file = os.path.join(main_folder, 'CSVFiles', os.path.split(path)[1] + '.csv')
    df = pd.DataFrame(columns = ['Image_name', 'Shoulder_L', 'Shoulder_R', 'Elbow_L', 'Elbow_R',
                                    'Waist_L', 'Waist_R', 'Knee_L', 'Knee_R', 'Pose_Num', 'Pose_Name'])
    pose_num = 0
        
    for pose_name in os.listdir(path):
        pose_folder = os.path.join(path, pose_name)

        for image in os.listdir(pose_folder):
            image_path = os.path.join(pose_folder, image)
            landmarks = get_landmarks(image_path, pose)
            if landmarks:
                features = get_features(landmarks)
                if None not in features:
                    row = pd.DataFrame({'Image_name' : image,
                                        'Shoulder_L' : features[0],
                                        'Shoulder_R' : features[1],
                                        'Elbow_L' : features[2],
                                        'Elbow_R' : features[3],
                                        'Waist_L' : features[4],
                                        'Waist_R' : features[5],
                                        'Knee_L' : features[6],
                                        'Knee_R' : features[7],
                                        'Pose_Num' : pose_num,
                                        'Pose_Name' : pose_list[pose_num]}, index=[0])
                    df = pd.concat([df, row], axis=0, ignore_index=True)            
            
        print(f"Successfully completed folder {pose_name}")
        pose_num += 1
    
    df.to_csv(csv_file, index=False)
    print(f"Successfully created {os.path.split(path)[1] + '.csv'}")

<font color='purple' size=4><b>Preprocessing Training Data</b></font>

In [8]:
preprocess('C:\\Users\\sumit\\Desktop\\yoga_trainer\\classification_model\\DATASET\\TRAIN')

Successfully completed folder downdog
Successfully completed folder goddess
Successfully completed folder plank
Successfully completed folder tree
Successfully completed folder warrior2
Successfully created TRAIN.csv


<font color='purple' size=4><b>Preprocessing Testing Data</b></font>

In [9]:
preprocess('C:\\Users\\sumit\\Desktop\\yoga_trainer\\classification_model\\DATASET\\TEST')

Successfully completed folder downdog
Successfully completed folder goddess
Successfully completed folder plank
Successfully completed folder tree
Successfully completed folder warrior2
Successfully created TEST.csv
